In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from datetime import datetime
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time

In [2]:
def comprobar_resultados(url, browser, fecha_limite, urls_provincias):
    urls_a_procesar = [url]
    
    while urls_a_procesar:
        url_actual = urls_a_procesar.pop()
        print(f"Procesando URL: {url_actual}")
        
        try:
            browser.get(url_actual)
            sleep(2)
            
            # Aceptar cookies
            try:
                browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
                sleep(1)
            except:
                pass
            
            # Extraer la provincia de la URL base
            provincia = url_actual.split('/')[-2]
            
            # Aplicar filtro de "Más recientes"
            try:
                select_element = browser.find_element(by=By.ID, value="sortResults")
                select = Select(select_element)
                filtro_value = f"/venta/{provincia}/fecharecientedesde-desc/"
                select.select_by_value(filtro_value)
                sleep(2)
            except Exception as e:
                print(f"Error al aplicar el filtro de 'Más recientes' en la URL {url_actual}: {e}")
                continue
            
            # Extraer URLs de los inmuebles
            inmuebles = browser.find_elements(by=By.CSS_SELECTOR, value="div.ad-preview")
            
            for i in range(len(inmuebles)):
                try:
                    inmuebles = browser.find_elements(by=By.CSS_SELECTOR, value="div.ad-preview")
                    inmueble = inmuebles[i]
                    link = inmueble.get_attribute('data-lnk-href')
                    full_link = f"https://www.pisos.com{link}"
                    
                    # Acceder a la URL del inmueble
                    browser.get(full_link)
                    sleep(2)
                    
                    # Extraer la fecha de primera publicación
                    span_element = browser.find_element(by=By.ID, value="vtmExtraVars")
                    data_var = span_element.get_attribute("data-var")
                    data = json.loads(data_var.replace('&quot;', '"'))
                    fecha_primera_publicacion_text = data["fechaPrimeraPublicacion"]
                    fecha_primera_publicacion = datetime.strptime(fecha_primera_publicacion_text, "%d/%m/%Y")
                    
                    if fecha_primera_publicacion >= fecha_limite:
                        provincia_nombre = provincia.replace('_', ' ').title()
                        if provincia_nombre not in urls_provincias:
                            urls_provincias[provincia_nombre] = []
                        urls_provincias[provincia_nombre].append(full_link)
                    
                    # Volver a la URL general
                    browser.back()
                    sleep(2)
                    
                    # Volver a aplicar el filtro de "Más recientes" después de regresar
                    try:
                        select_element = browser.find_element(by=By.ID, value="sortResults")
                        select = Select(select_element)
                        select.select_by_value(filtro_value)
                        sleep(2)
                    except Exception as e:
                        print(f"Error al reaplicar el filtro de 'Más recientes' en la URL {url_actual}: {e}")
                        return
                except Exception as e:
                    print(f"Error al procesar el inmueble en la URL {full_link}: {e}")
                    continue
        
        except Exception as e:
            print(f"Error al procesar la URL {url_actual}: {e}")

# Inicializar el navegador
browser = webdriver.Chrome()
browser.maximize_window()

# Lista de URLs iniciales
urls = [
    "https://www.pisos.com/venta/pisos-madrid/",
    "https://www.pisos.com/venta/pisos-la_rioja/",
    "https://www.pisos.com/venta/pisos-navarra_nafarroa/",
    "https://www.pisos.com/venta/pisos-asturias/",
    "https://www.pisos.com/venta/pisos-cantabria/",
    "https://www.pisos.com/venta/pisos-murcia/",
    "https://www.pisos.com/venta/pisos-islas_baleares_illes_balears/",
    "https://www.pisos.com/venta/pisos-las_palmas/",
    "https://www.pisos.com/venta/pisos-santa_cruz_de_tenerife/",
    "https://www.pisos.com/venta/pisos-badajoz/",
    "https://www.pisos.com/venta/pisos-caceres/",
    "https://www.pisos.com/venta/pisos-guipuzcoa_gipuzkoa/",
    "https://www.pisos.com/venta/pisos-vizcaya_bizkaia/",
    "https://www.pisos.com/venta/pisos-castellon_castello/",
    "https://www.pisos.com/venta/pisos-valencia/",
    "https://www.pisos.com/venta/pisos-alicante/",
    "https://www.pisos.com/venta/pisos-teruel/",
    "https://www.pisos.com/venta/pisos-huesca/",
    "https://www.pisos.com/venta/pisos-zaragoza/",
    "https://www.pisos.com/venta/pisos-girona/",
    "https://www.pisos.com/venta/pisos-lleida/",
    "https://www.pisos.com/venta/pisos-barcelona/",
    "https://www.pisos.com/venta/pisos-lugo/",
    "https://www.pisos.com/venta/pisos-pontevedra/",
    "https://www.pisos.com/venta/pisos-ourense/",
    "https://www.pisos.com/venta/pisos-a_coruna/",
    "https://www.pisos.com/venta/pisos-guadalajara/",
    "https://www.pisos.com/venta/pisos-toledo/",
    "https://www.pisos.com/venta/pisos-cuenca/",
    "https://www.pisos.com/venta/pisos-albacete/",
    "https://www.pisos.com/venta/pisos-ciudad_real/",
    "https://www.pisos.com/venta/pisos-salamanca/",
    "https://www.pisos.com/venta/pisos-avila/",
    "https://www.pisos.com/venta/pisos-segovia/",
    "https://www.pisos.com/venta/pisos-soria/",
    "https://www.pisos.com/venta/pisos-burgos/",
    "https://www.pisos.com/venta/pisos-palencia/",
    "https://www.pisos.com/venta/pisos-valladolid/",
    "https://www.pisos.com/venta/pisos-leon/",
    "https://www.pisos.com/venta/pisos-zamora/",
    "https://www.pisos.com/venta/pisos-alava_araba/",
    "https://www.pisos.com/venta/pisos-almeria/",
    "https://www.pisos.com/venta/pisos-cadiz/",
    "https://www.pisos.com/venta/pisos-castellon_castello/",
    "https://www.pisos.com/venta/pisos-cordoba/",
    "https://www.pisos.com/venta/pisos-granada/",
    "https://www.pisos.com/venta/pisos-huelva/",
    "https://www.pisos.com/venta/pisos-jaen/",
    "https://www.pisos.com/venta/pisos-malaga/",
    "https://www.pisos.com/venta/pisos-sevilla/",
    "https://www.pisos.com/venta/pisos-tarragona/",
    "https://www.pisos.com/venta/pisos-ceuta/",
    "https://www.pisos.com/venta/pisos-melilla/",
    "https://www.pisos.com/venta/pisos-islas_baleares_illes_balears/"
]

# Fecha límite para filtrar los inmuebles
fecha_limite = datetime.strptime("2025-03-01", "%Y-%m-%d")

urls_provincias = {}

# Recorrer las URLs iniciales
for url in urls:
    comprobar_resultados(url, browser, fecha_limite, urls_provincias)

# Imprimir las URLs de los pisos nuevos por provincia
print("URLs de los pisos nuevos publicados después de la fecha límite por provincia:")
for provincia, urls in urls_provincias.items():
    print(f"{provincia}:")
    for url in urls:
        print(f"  - {url}")

# Cerrar el navegador después de procesar todas las URLs
browser.quit()

Procesando URL: https://www.pisos.com/venta/pisos-madrid/
Procesando URL: https://www.pisos.com/venta/pisos-la_rioja/
Procesando URL: https://www.pisos.com/venta/pisos-navarra_nafarroa/
Procesando URL: https://www.pisos.com/venta/pisos-asturias/
Procesando URL: https://www.pisos.com/venta/pisos-cantabria/
Procesando URL: https://www.pisos.com/venta/pisos-murcia/
Procesando URL: https://www.pisos.com/venta/pisos-islas_baleares_illes_balears/
Procesando URL: https://www.pisos.com/venta/pisos-las_palmas/
Procesando URL: https://www.pisos.com/venta/pisos-santa_cruz_de_tenerife/
Procesando URL: https://www.pisos.com/venta/pisos-badajoz/
Procesando URL: https://www.pisos.com/venta/pisos-caceres/
Procesando URL: https://www.pisos.com/venta/pisos-guipuzcoa_gipuzkoa/
Procesando URL: https://www.pisos.com/venta/pisos-vizcaya_bizkaia/
Procesando URL: https://www.pisos.com/venta/pisos-castellon_castello/
Procesando URL: https://www.pisos.com/venta/pisos-valencia/
Procesando URL: https://www.pisos.

In [43]:
#labuena
def comprobar_resultados(url, browser, fecha_limite, urls_provincias):
    urls_a_procesar = [url]
    
    while urls_a_procesar:
        url_actual = urls_a_procesar.pop()
        print(f"Procesando URL: {url_actual}")
        
        try:
            browser.get(url_actual)
            sleep(2)
            
            # Aceptar cookies
            try:
                browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
                sleep(1)
            except:
                pass
            
            # Extraer la provincia de la URL base
            provincia = url_actual.split('/')[-2]
            
            # Aplicar filtro de "Más recientes"
            try:
                select_element = browser.find_element(by=By.ID, value="sortResults")
                select = Select(select_element)
                filtro_value = f"/venta/{provincia}/fecharecientedesde-desc/"
                select.select_by_value(filtro_value)
                sleep(2)
            except Exception as e:
                print(f"Error al aplicar el filtro de 'Más recientes' en la URL {url_actual}: {e}")
                continue
            
            # Extraer URLs de los inmuebles
            inmuebles = browser.find_elements(by=By.CSS_SELECTOR, value="div.ad-preview")
            
            for i in range(len(inmuebles)):
                try:
                    inmuebles = browser.find_elements(by=By.CSS_SELECTOR, value="div.ad-preview")
                    inmueble = inmuebles[i]
                    link = inmueble.get_attribute('data-lnk-href')
                    full_link = f"https://www.pisos.com{link}"
                    
                    # Acceder a la URL del inmueble
                    browser.get(full_link)
                    sleep(2)
                    
                    # Extraer la fecha de primera publicación
                    span_element = browser.find_element(by=By.ID, value="vtmExtraVars")
                    data_var = span_element.get_attribute("data-var")
                    data = json.loads(data_var.replace('&quot;', '"'))
                    fecha_primera_publicacion_text = data["fechaPrimeraPublicacion"]
                    fecha_primera_publicacion = datetime.strptime(fecha_primera_publicacion_text, "%d/%m/%Y")
                    
                    if fecha_primera_publicacion >= fecha_limite:
                        provincia_nombre = provincia.replace('_', ' ').title()
                        if provincia_nombre not in urls_provincias:
                            urls_provincias[provincia_nombre] = []
                        urls_provincias[provincia_nombre].append(full_link)
                    
                    # Volver a la URL general
                    browser.back()
                    sleep(2)
                    
                    # Volver a aplicar el filtro de "Más recientes" después de regresar
                    try:
                        select_element = browser.find_element(by=By.ID, value="sortResults")
                        select = Select(select_element)
                        select.select_by_value(filtro_value)
                        sleep(2)
                    except Exception as e:
                        print(f"Error al reaplicar el filtro de 'Más recientes' en la URL {url_actual}: {e}")
                        return
                except Exception as e:
                    print(f"Error al procesar el inmueble en la URL {full_link}: {e}")
                    continue
        
        except Exception as e:
            print(f"Error al procesar la URL {url_actual}: {e}")

# Inicializar el navegador
browser = webdriver.Chrome()
browser.maximize_window()

# Lista de URLs iniciales
urls = [
    "https://www.pisos.com/venta/pisos-madrid/",
    "https://www.pisos.com/venta/pisos-la_rioja/",
    "https://www.pisos.com/venta/pisos-navarra_nafarroa/",
    "https://www.pisos.com/venta/pisos-asturias/",
    "https://www.pisos.com/venta/pisos-cantabria/",
    "https://www.pisos.com/venta/pisos-murcia/",
    "https://www.pisos.com/venta/pisos-islas_baleares_illes_balears/",
    "https://www.pisos.com/venta/pisos-las_palmas/",
    "https://www.pisos.com/venta/pisos-santa_cruz_de_tenerife/",
    "https://www.pisos.com/venta/pisos-badajoz/",
    "https://www.pisos.com/venta/pisos-caceres/",
    "https://www.pisos.com/venta/pisos-guipuzcoa_gipuzkoa/",
    "https://www.pisos.com/venta/pisos-vizcaya_bizkaia/",
    "https://www.pisos.com/venta/pisos-castellon_castello/",
    "https://www.pisos.com/venta/pisos-valencia/",
    "https://www.pisos.com/venta/pisos-alicante/",
    "https://www.pisos.com/venta/pisos-teruel/",
    "https://www.pisos.com/venta/pisos-huesca/",
    "https://www.pisos.com/venta/pisos-zaragoza/",
    "https://www.pisos.com/venta/pisos-girona/",
    "https://www.pisos.com/venta/pisos-lleida/",
    "https://www.pisos.com/venta/pisos-barcelona/",
    "https://www.pisos.com/venta/pisos-lugo/",
    "https://www.pisos.com/venta/pisos-pontevedra/",
    "https://www.pisos.com/venta/pisos-ourense/",
    "https://www.pisos.com/venta/pisos-a_coruna/",
    "https://www.pisos.com/venta/pisos-guadalajara/",
    "https://www.pisos.com/venta/pisos-toledo/",
    "https://www.pisos.com/venta/pisos-cuenca/",
    "https://www.pisos.com/venta/pisos-albacete/",
    "https://www.pisos.com/venta/pisos-ciudad_real/",
    "https://www.pisos.com/venta/pisos-salamanca/",
    "https://www.pisos.com/venta/pisos-avila/",
    "https://www.pisos.com/venta/pisos-segovia/",
    "https://www.pisos.com/venta/pisos-soria/",
    "https://www.pisos.com/venta/pisos-burgos/",
    "https://www.pisos.com/venta/pisos-palencia/",
    "https://www.pisos.com/venta/pisos-valladolid/",
    "https://www.pisos.com/venta/pisos-leon/",
    "https://www.pisos.com/venta/pisos-zamora/",
    "https://www.pisos.com/venta/pisos-alava_araba/",
    "https://www.pisos.com/venta/pisos-almeria/",
    "https://www.pisos.com/venta/pisos-cadiz/",
    "https://www.pisos.com/venta/pisos-castellon_castello/",
    "https://www.pisos.com/venta/pisos-cordoba/",
    "https://www.pisos.com/venta/pisos-granada/",
    "https://www.pisos.com/venta/pisos-huelva/",
    "https://www.pisos.com/venta/pisos-jaen/",
    "https://www.pisos.com/venta/pisos-malaga/",
    "https://www.pisos.com/venta/pisos-sevilla/",
    "https://www.pisos.com/venta/pisos-tarragona/",
    "https://www.pisos.com/venta/pisos-ceuta/",
    "https://www.pisos.com/venta/pisos-melilla/",
    "https://www.pisos.com/venta/pisos-islas_baleares_illes_balears/"
]

# Fecha límite para filtrar los inmuebles
fecha_limite = datetime.strptime("2025-03-01", "%Y-%m-%d")

urls_provincias = {}

# Recorrer las URLs iniciales
for url in urls:
    comprobar_resultados(url, browser, fecha_limite, urls_provincias)

# Guardar el output en un archivo de texto en el formato especificado
with open('urls_provincias_output.txt', 'w') as file:
    for provincia, urls in urls_provincias.items():
        for url in urls:
            file.write(f"{url} ({provincia})\n")

print("El output ha sido guardado en urls_provincias_output.txt")

# Cerrar el navegador después de procesar todas las URLs
browser.quit()

print("Las URLs han sido guardadas en urls_provincias_output.txt")

Procesando URL: https://www.pisos.com/venta/pisos-madrid/
Error al reaplicar el filtro de 'Más recientes' en la URL https://www.pisos.com/venta/pisos-madrid/: 'NoneType' object has no attribute 'tag_name'
Procesando URL: https://www.pisos.com/venta/pisos-la_rioja/
Error al procesar la URL https://www.pisos.com/venta/pisos-la_rioja/: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.142)
Stacktrace:
0   chromedriver                        0x00000001052082d4 cxxbridge1$str$ptr + 2739836
1   chromedriver                        0x0000000105200934 cxxbridge1$str$ptr + 2708700
2   chromedriver                        0x0000000104d61f90 cxxbridge1$string$len + 93360
3   chromedriver                        0x0000000104d3b964 chromedriver + 129380
4   chromedriver                        0x0000000104dd0edc cxxbridge1$string$len + 547836
5   chromedriver                        0x0000000104de9994 cxxbridge1$string$len + 6

In [3]:
# Función para verificar si una página existe
def pagina_existe(url):
    response = requests.get(url)
    if response.status_code != 200:
        return False
    soup = BeautifulSoup(response.text, 'html.parser')

    no_results = soup.find('div', class_='no-results')
    if no_results:
        return False
    return True

In [ ]:
# Función para obtener detalles de un inmueble
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        try:
            nombre = soup.find("h1", class_="ad-title")
            nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        except:
            nombre = "Error al acceder al inmueble"

        try:
            agencia = soup.find("p", class_="owner-info__name")
            agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"
        except:
            agencia = "Agencia no disponible"

        try:
            features_summary = soup.find("ul", class_="features-summary")
            precio_m2 = None
            if features_summary:
                items = features_summary.find_all("li", class_="features-summary__item")
                for item in items:
                    if "€/m²" in item.text:
                        precio_m2 = item.text.strip()
                        break
            precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        except:
            precio_m2 = "Error al procesar precio m²"
        
        try:
            precio = soup.find("div", class_="price__value jsPriceValue")
            precio = precio.text.strip() if precio else "Error al procesar precio"
        except:
            precio = "Error al procesar precio"
        
        try:
            superficie = soup.find("span", class_="features__value")
            superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        except:
            superficie = "Error al procesar superficie"
        
        try:
            actualizacion = soup.find("div", class_="details__block last-update")
            actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        except:
            actualizacion = "Error al procesar actualizacion"

        try:
            consumo = soup.find_all("span", class_="energy-certificate__tag")
            consumo_etiquetas = []
            for etiqueta in consumo:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                    if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                        consumo_etiquetas.append(letra_consumo.upper())
            consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"
        except:
            consumo = "Error al procesar consumo"

        try:
            emisiones = soup.find_all("span", class_="energy-certificate__tag")
            emisiones_etiquetas = []
            for etiqueta in emisiones:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                    if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                        emisiones_etiquetas.append(letra_emisiones.upper())
            emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"
        except:
            emisiones = "Error al procesar emisiones"

        try:
            caracteristicas = soup.find("div", class_="features-container")
            detalles = {}
            if caracteristicas:
                secciones = caracteristicas.find_all("div", class_="features__content")
                for seccion in secciones:
                    features = seccion.find_all("div", class_="features__feature")
                    for feature in features:
                        label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                        value = feature.find("span", class_="features__value").text.strip() if feature.find("span", "features__value") else "Si"
                        detalles[label] = value
        except:
            detalles = {}

        # Aquí vamos a extraer el Código Postal e Identificador mediante la URL de cada inmueble
        try:
            codigo_postal_match = re.search(r'(\d{5})-', url_inmueble)
            codigo_postal = codigo_postal_match.group(1) if codigo_postal_match else 'NaN'
        except Exception as e:
            codigo_postal = f"Error al extraer código postal: {e}"
        
        try:
            identificador_match = re.search(r'-(\d+_\d+)', url_inmueble)
            identificador = identificador_match.group(1) if identificador_match else 'NaN'
        except Exception as e:
            identificador = f"Error al extraer identificador: {e}"
        
        # Aquí extraemos el timestamp de cada extracción de datos, que nos será útil a la hora de actualizar la db
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Aquí extraemos las coordenadas (latitud y longitud)
        try:
            location_div = soup.find("div", class_="location")
            data_params = location_div["data-params"]
            
            lat_match = re.search(r'latitude=([-0-9.]+)', data_params)
            lon_match = re.search(r'longitude=([-0-9.]+)', data_params)
            
            latitud = lat_match.group(1) if lat_match else 'NaN'
            longitud = lon_match.group(1) if lon_match else 'NaN'
            
            coordenadas = f"{latitud}, {longitud}"
        except Exception as e:
            coordenadas = f"Error al extraer coordenadas: {e}"
            
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas
    except Exception as e:
        return ("Error al acceder al inmueble",) * 9 + ({}, "", "", datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "")

In [88]:
# Función para verificar si una página existe
def pagina_existe(url):
    response = requests.get(url)
    if response.status_code != 200:
        return False
    soup = BeautifulSoup(response.text, 'html.parser')

    no_results = soup.find('div', class_='no-results')
    if no_results:
        return False
    return True

In [4]:
# Función para obtener detalles de un inmueble
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        try:
            nombre = soup.find("h1", class_="ad-title")
            nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        except:
            nombre = "Error al acceder al inmueble"

        try:
            agencia = soup.find("p", class_="owner-info__name")
            agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"
        except:
            agencia = "Agencia no disponible"

        try:
            features_summary = soup.find("ul", class_="features-summary")
            precio_m2 = None
            if features_summary:
                items = features_summary.find_all("li", class_="features-summary__item")
                for item in items:
                    if "€/m²" in item.text:
                        precio_m2 = item.text.strip()
                        break
            precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        except:
            precio_m2 = "Error al procesar precio m²"
        
        try:
            precio = soup.find("div", class_="price__value jsPriceValue")
            precio = precio.text.strip() if precio else "Error al procesar precio"
        except:
            precio = "Error al procesar precio"
        
        try:
            superficie = soup.find("span", class_="features__value")
            superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        except:
            superficie = "Error al procesar superficie"
        
        try:
            actualizacion = soup.find("div", class_="details__block last-update")
            actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        except:
            actualizacion = "Error al procesar actualizacion"

        try:
            consumo = soup.find_all("span", class_="energy-certificate__tag")
            consumo_etiquetas = []
            for etiqueta in consumo:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                    if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                        consumo_etiquetas.append(letra_consumo.upper())
            consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"
        except:
            consumo = "Error al procesar consumo"

        try:
            emisiones = soup.find_all("span", class_="energy-certificate__tag")
            emisiones_etiquetas = []
            for etiqueta in emisiones:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                    if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                        emisiones_etiquetas.append(letra_emisiones.upper())
            emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"
        except:
            emisiones = "Error al procesar emisiones"

        try:
            caracteristicas = soup.find("div", class_="features-container")
            detalles = {}
            if caracteristicas:
                secciones = caracteristicas.find_all("div", class_="features__content")
                for seccion in secciones:
                    features = seccion.find_all("div", class_="features__feature")
                    for feature in features:
                        label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                        value = feature.find("span", class_="features__value").text.strip() if feature.find("span", "features__value") else "Si"
                        detalles[label] = value
        except:
            detalles = {}

        # Aquí vamos a extraer el Código Postal e Identificador mediante la URL de cada inmueble
        try:
            codigo_postal_match = re.search(r'(\d{5})-', url_inmueble)
            codigo_postal = codigo_postal_match.group(1) if codigo_postal_match else 'NaN'
        except Exception as e:
            codigo_postal = f"Error al extraer código postal: {e}"
        
        try:
            identificador_match = re.search(r'-(\d+_\d+)', url_inmueble)
            identificador = identificador_match.group(1) if identificador_match else 'NaN'
        except Exception as e:
            identificador = f"Error al extraer identificador: {e}"
        
        # Aquí extraemos el timestamp de cada extracción de datos, que nos será útil a la hora de actualizar la db
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Aquí extraemos las coordenadas (latitud y longitud)
        try:
            location_div = soup.find("div", class_="location")
            data_params = location_div["data-params"]
            
            lat_match = re.search(r'latitude=([-0-9.]+)', data_params)
            lon_match = re.search(r'longitude=([-0-9.]+)', data_params)
            
            latitud = lat_match.group(1) if lat_match else 'NaN'
            longitud = lon_match.group(1) if lon_match else 'NaN'
            
            coordenadas = f"{latitud}, {longitud}"
        except Exception as e:
            coordenadas = f"Error al extraer coordenadas: {e}"
            
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas
    except Exception as e:
        return ("Error al acceder al inmueble",) * 9 + ({}, "", "", datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "")

In [41]:
def obtener_inmuebles_de_urls(urls_a_scrapear, ubicaciones):
    todos_los_inmuebles = []
    columnas = set()

    for url_pagina, ubicacion in zip(urls_a_scrapear, ubicaciones):
        print(f"Procesando la URL: {url_pagina}")
        
        try:
            response = requests.get(url_pagina)
            if response.status_code == 404:
                print(f"Página no encontrada: {url_pagina}")
                continue
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Ajustar el selector CSS según la estructura actual del HTML
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            if not titulos_soup:
                titulos_soup = soup.find_all("h2", class_="title")  # Ejemplo de otro posible selector
            print(f"Encontrados {len(titulos_soup)} títulos en la página.")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                print(f"Procesando inmueble: {nombre}, URL: {href}")
                nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas = obtener_detalles_inmueble(href)
                
                inmueble = {
                    "nombre": nombre,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "consumo": consumo,
                    "emisiones": emisiones,
                    "ubicacion": ubicacion,
                    "codigo_postal": codigo_postal,
                    "identificador": identificador,
                    "timestamp": timestamp,
                    "coordenadas": coordenadas
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
                columnas.update(inmueble.keys())
        
        except Exception as e:
            print(f"Error al procesar la URL {url_pagina}: {e}")
        time.sleep(1) 
    
    df_inmuebles = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_inmuebles

In [42]:
file_path = 'urls_test.txt'

try:
    with open(file_path, 'r') as file:
        urls = file.read().splitlines()
    print(f"Se han cargado {len(urls)} URLs desde el archivo.")
    
    urls_limpias = []
    ubicaciones = []

    for url in urls:
        # Eliminar la parte después de la URL
        url_limpia = url.split(' ')[0]
        urls_limpias.append(url_limpia)
        # Extraer la ubicación desde los paréntesis
        ubicacion = url.split('(')[-1].split(')')[0].split('-')[-1]
        ubicaciones.append(ubicacion)
        print(f"Procesando la URL: {url_limpia}")
        print(f"Ubicación extraída: {ubicacion}")
    
    # Obtener inmuebles de las URLs generadas
    df_inmuebles = obtener_inmuebles_de_urls(urls_limpias, ubicaciones)
except FileNotFoundError:
    print(f"El archivo {file_path} no se encontró.")

Se han cargado 5 URLs desde el archivo.
Procesando la URL: https://www.pisos.com/comprar/piso-getafe_el_bercial28907-52567587798_100700/
Ubicación extraída: Madrid
Procesando la URL: https://www.pisos.com/comprar/piso-sevilla_la_nueva_centro_urbano-52522689677_100700/
Ubicación extraída: Madrid
Procesando la URL: https://www.pisos.com/comprar/piso-recoletos28001-52582039160_108500/
Ubicación extraída: Madrid
Procesando la URL: https://www.pisos.com/comprar/duplex-ciudad_lineal_colina28043-52580671500_108500/
Ubicación extraída: Madrid
Procesando la URL: https://www.pisos.com/comprar/piso-moncloa_aravaca_arguelles28008-52520692555_108500/
Ubicación extraída: Madrid
Procesando la URL: https://www.pisos.com/comprar/piso-getafe_el_bercial28907-52567587798_100700/
Encontrados 0 títulos en la página.
Procesando la URL: https://www.pisos.com/comprar/piso-sevilla_la_nueva_centro_urbano-52522689677_100700/
Encontrados 0 títulos en la página.
Procesando la URL: https://www.pisos.com/comprar/piso

In [38]:
df_inmuebles

""


In [90]:
# Función para extraer la ubicación de la cadena "Pisos-{ubicacion}:"
def extract_location(text):
    return text.split('-')[-1].split(':')[0]

In [91]:
# Función para obtener inmuebles de URLs
def obtener_inmuebles_de_urls(urls_a_scrapear):
    todos_los_inmuebles = []
    columnas = set()

    for url_pagina, ubicacion in urls_a_scrapear:
        url_pagina = url_pagina.strip('- ').strip()  # Eliminamos los guiones generados en la lista
        print(f"Procesando la URL: {url_pagina} con ubicación: {ubicacion}")
        
        try:
            response = requests.get(url_pagina)
            if response.status_code == 404:
                print(f"Página no encontrada: {url_pagina}")
                continue
            soup = BeautifulSoup(response.text, "html.parser")
            
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas = obtener_detalles_inmueble(href)
                
                inmueble = {
                    "nombre": nombre,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "consumo": consumo,
                    "emisiones": emisiones,
                    "ubicacion": ubicacion,  # Asegúrate de que la ubicación se asigne aquí
                    "codigo_postal": codigo_postal,
                    "identificador": identificador,
                    "timestamp": timestamp,
                    "coordenadas": coordenadas
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
                columnas.update(inmueble.keys())
                print(f"Inmueble añadido: {inmueble}")  # Agregado para depuración
        
        except Exception as e:
            print(f"Error al procesar la URL {url_pagina}: {e}")
        time.sleep(1) 
    
    df_inmuebles = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    print(f"DataFrame final: {df_inmuebles}")  # Agregado para depuración
    return df_inmuebles

In [92]:
# Leer el archivo txt con las URLs y asignarlas a la lista 'urls'
file_path = 'urls_test.txt'
try:
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
        ubicacion = None
        urls_nuevas = []
        for line in lines:
            if line.startswith("Pisos-"):
                ubicacion = extract_location(line)
                print(f"Ubicación encontrada: {ubicacion}")
            elif line.strip():
                urls_nuevas.append((line.strip(), ubicacion))
    print(f"Se han cargado {len(urls_nuevas)} URLs desde el archivo.")
    print(f"URLs y ubicaciones: {urls_nuevas}")  # Agregado para depuración
except FileNotFoundError:
    print(f"El archivo {file_path} no se encontró.")

Ubicación encontrada: Madrid
Ubicación encontrada: La Rioja
Ubicación encontrada: Navarra Nafarroa
Ubicación encontrada: Asturias
Se han cargado 8 URLs desde el archivo.
URLs y ubicaciones: [('- https://www.pisos.com/comprar/piso-somosaguas_humera_los_angeles28223-52555716911_102100/', 'Madrid'), ('- https://www.pisos.com/comprar/piso-pryconsa_juan_de_austria28805-52575993637_100500/', 'Madrid'), ('- https://www.pisos.com/comprar/atico-villamediana_de_iregua_centro_urbano-52521527556_102200/', 'La Rioja'), ('- https://www.pisos.com/comprar/apartamento-universidad_los_lirios26004-52549840254_991833/', 'La Rioja'), ('- https://www.pisos.com/comprar/piso-baranain_baranain_centro_urbano-52595892853_510843/', 'Navarra Nafarroa'), ('- https://www.pisos.com/comprar/piso-berriozar_centro_urbano-52570268287_109800/', 'Navarra Nafarroa'), ('- https://www.pisos.com/comprar/chalet-villaviciosa_centro_urbano-52558070209_100500/', 'Asturias'), ('- https://www.pisos.com/comprar/piso-otero_villafria_s

In [93]:
# Obtener inmuebles de las URLs generadas y la ubicacion
df_inmuebles = obtener_inmuebles_de_urls(urls_nuevas)

Procesando la URL: https://www.pisos.com/comprar/piso-somosaguas_humera_los_angeles28223-52555716911_102100/ con ubicación: Madrid
Procesando la URL: https://www.pisos.com/comprar/piso-pryconsa_juan_de_austria28805-52575993637_100500/ con ubicación: Madrid
Procesando la URL: https://www.pisos.com/comprar/atico-villamediana_de_iregua_centro_urbano-52521527556_102200/ con ubicación: La Rioja
Procesando la URL: https://www.pisos.com/comprar/apartamento-universidad_los_lirios26004-52549840254_991833/ con ubicación: La Rioja
Procesando la URL: https://www.pisos.com/comprar/piso-baranain_baranain_centro_urbano-52595892853_510843/ con ubicación: Navarra Nafarroa
Procesando la URL: https://www.pisos.com/comprar/piso-berriozar_centro_urbano-52570268287_109800/ con ubicación: Navarra Nafarroa
Inmueble añadido: {'nombre': 'Piso en Berriozar', 'agencia': 'MOUNTAIN VIEW REAL ESTATE BURGOS', 'precio_m2': '2.121 €/m²', 'precio': '237.600 €', 'superficie': '112 m²', 'href': 'https://www.pisos.com/comp

In [85]:
df_inmuebles.columns

Index([], dtype='object')

In [86]:
df_inmuebles.shape

(0, 0)

In [87]:
df_inmuebles["ubicacion"]

KeyError: 'ubicacion'